In [3]:
import polars as pl

full_df = pl.read_csv('PI1M.csv')
full_df.head()

SMILES
str
"""*CCC[Fe]CCCC(=O)OCCCCOCCCNCC(*…"
"""*CCCC1C=CNC2=CC=C2C(*)CCC1"""
"""*CCC*"""
"""*C(=O)CNC(*)C(=O)OCCCCCNC"""
"""*CC(C)(C)CCCCCCC(C)C(=O)N*"""


In [4]:
from rdkit import Chem
from tqdm import tqdm

full_subsample_df = full_df.sample(n=100000, with_replacement=False)
filtered_smiles = []
for smiles in tqdm(full_subsample_df['SMILES']):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        filtered_smiles.append(smiles)

  0%|          | 0/100000 [00:00<?, ?it/s][23:31:12] Explicit valence for atom # 2 P, 6, is greater than permitted
[23:31:12] Explicit valence for atom # 22 P, 7, is greater than permitted
[23:31:12] Explicit valence for atom # 2 P, 7, is greater than permitted
[23:31:12] Explicit valence for atom # 2 P, 6, is greater than permitted
  1%|          | 1021/100000 [00:00<00:19, 5128.56it/s][23:31:12] Explicit valence for atom # 16 P, 7, is greater than permitted
[23:31:12] Explicit valence for atom # 2 P, 7, is greater than permitted
[23:31:12] Explicit valence for atom # 9 P, 6, is greater than permitted
[23:31:12] Explicit valence for atom # 4 P, 6, is greater than permitted
[23:31:12] Explicit valence for atom # 5 P, 6, is greater than permitted
[23:31:12] Explicit valence for atom # 2 P, 6, is greater than permitted
  2%|▏         | 2041/100000 [00:00<00:20, 4710.65it/s][23:31:12] Explicit valence for atom # 12 P, 7, is greater than permitted
[23:31:12] Explicit valence for atom # 16 

In [5]:
filtered_df = full_subsample_df.filter(pl.col('SMILES').is_in(filtered_smiles))
filtered_df.shape

(99278, 1)

In [6]:
# RECORD_COUNT = 50000

# subsampled_df = filtered_df.sample(n=RECORD_COUNT, with_replacement=False)
# subsampled_df.write_csv(f'PI1M_{RECORD_COUNT}.csv')

In [8]:
import polars as pl

blacklist_df = pl.read_csv('PI1M_50000.csv')
blacklist_smiles = set(blacklist_df['SMILES'])

unique_filtered_smiles = [smi for smi in filtered_smiles if smi not in blacklist_smiles]
rows = ['SMILES'] + unique_filtered_smiles

with open('PI1M_50000_rankup.csv', 'w') as output_file:
    output_file.write('\n'.join(rows[:50001]))  # Write only the first 50,000 rows